<a href="https://colab.research.google.com/github/yukilost/googlecolab/blob/main/LearningBasedFluids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>GraphNeruralNetworkを用いた非圧縮性流体の教師なし学習</h1>

#初期設定

##ライブラリのインポート

In [2]:
from google.colab import drive
import os
import math
import pandas as pd
import numpy as np
from sklearn import neighbors
import torch
import torch.nn as nn

##GoogleDriveのマウント

In [2]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##デバイス設定

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device: {}".format(device))

device: cpu


#データセット

In [ ]:
!unzip /content/drive/MyDrive/LearningBasedFluids/data.zip -d /content/graph_neural_network/

#グラフニューラルネットワーク

In [ ]:
class Graph:
  def __init__(self, pos, vel, vol):
    self.pos = pos
    self.vel = vel
    self.vol = vol

    indicies = NearestNeighborSearch(pos)

    receivers = np.zeros(0, dtype=np.int)
    senders = np.zeros(0, dtype=np.int)
    
    for receiver, neighbor in enumerate(neighbor_indices):
      for sender in neighbor:
        # 近傍粒子から自身を除く
        if sender == receiver: continue
        receivers = np.append(receivers, receiver)
        senders = np.append(senders, sender)
    
    relative_pos = pos[receivers.astype(np.int)] - pos[senders.astype(np.int)]
    magnitude = np.linalg.norm(relative_pos, axis=1).reshape(-1, 1)
    

  def NearestNeighborSearch(self, pos):


In [4]:
def calcParticleRadius(config):
  fluid_particle_num = config["Simulation"]["FluidSize"]["x"] * config["Simulation"]["FluidSize"]["y"] * config["Simulation"]["FluidSize"]["z"]
  particle_mass = config["Simulation"]["ParticleMass"]
  rest_density = config["Simulation"]["RestDensity"]
  kernel_particles = config["Simulation"]["KernelParticles"]

  volume = fluid_particle_num * particle_mass / rest_density;

  effective_radius = pow(((3.0*kernel_particles*volume)/(4.0*fluid_particle_num*math.pi)), 1.0/3.0);
  particle_radius = pow((math.pi/(6.0*kernel_particles)), 1.0/3.0)*effective_radius;

  return particle_radius

In [6]:
# TODO: leaf_sizeについて調べる
# TODO: ハッシュ値を使った実装に変更

def NearestNeighborSearch(pos, radius):
  tree = neighbors.KDTree(pos, leaf_size=2)
  idx = tree.query_radius(pos[:], r=radius).tolist()
  return idx

In [7]:
# TODO 正規化

class GraphNeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()

    # Encoder
    self.epsilon_v = nn.Sequential(
        nn.Linear(in_features=13, out_features=128), nn.ReLU(), 
        nn.Linear(in_features=128, out_features=128), nn.ReLU(),
        nn.Linear(in_features=128, out_features=128), nn.ReLU()
    )
    self.epsilon_e = nn.Sequential(
        nn.Linear(in_features=4, out_features=128), nn.ReLU(), 
        nn.Linear(in_features=128, out_features=128), nn.ReLU(),
        nn.Linear(in_features=128, out_features=128), nn.ReLU(),
    )

    # Processor
    self.phi_v = nn.Sequential(
        nn.Linear(in_features=128*2, out_features=128), nn.ReLU(), 
        nn.Linear(in_features=128, out_features=128), nn.ReLU(),
        nn.Linear(in_features=128, out_features=128), nn.ReLU()
    )
    self.phi_e = nn.Sequential(
        nn.Linear(in_features=128*3, out_features=128), nn.ReLU(), 
        nn.Linear(in_features=128, out_features=128), nn.ReLU(),
        nn.Linear(in_features=128, out_features=128), nn.ReLU()
    )

    # Decoder
    self.delta_v = nn.Sequential(
        nn.Linear(in_features=128, out_features=128), nn.ReLU(), 
        nn.Linear(in_features=128, out_features=128), nn.ReLU(), 
        nn.Linear(in_features=128, out_features=3)
    )
  
  def forward(self, df, radius, M, config):
    pos, vel, v, receivers, senders, _, _, e = self.encode(df, radius, config)
    self.process(v, receivers, senders, e, M)
    a = self.decode(v)
    return a

  def encode(self, df, radius, config):
    pos = df[["position.x", "position.y", "position.z"]].values
    vel = df[["velocity.x", "velocity.y", "velocity.z"]].values
    vol = df[["volume"]].values
    
    particle_radius = calcParticleRadius(config)
    bx = config["Simulation"]["BoundarySize"]["x"]
    by = config["Simulation"]["BoundarySize"]["y"]
    bz = config["Simulation"]["BoundarySize"]["z"]
    
    x1 = 2.0*particle_radius
    y1 = 2.0*particle_radius
    z1 = 2.0*particle_radius
    x2 = (bx-1)*2.0*particle_radius
    y2 = (by-1)*2.0*particle_radius
    z2 = (bz-1)*2.0*particle_radius
    
    x1 = abs(x1-pos[:, 0]).reshape((-1, 1))
    x2 = abs(x2-pos[:, 0]).reshape((-1, 1))
    y1 = abs(y1-pos[:, 1]).reshape((-1, 1))
    y2 = abs(y2-pos[:, 1]).reshape((-1, 1))
    z1 = abs(z1-pos[:, 2]).reshape((-1, 1))
    z2 = abs(z2-pos[:, 2]).reshape((-1, 1))
    
    # v = np.concatenate((pos, vel), axis=1).astype(np.float32)
    g = np.array([[0.0, -9.8, 0.0]]*len(df))
    v = np.concatenate((vel, vol, g, x1, x2, y1, y2, z1, z2), axis=1).astype(np.float32)
    v = torch.from_numpy(v).clone()
    v = self.epsilon_v(v)
    
    neighbor_indices = NearestNeighborSearch(pos, radius)
    receivers = np.zeros(0, dtype=np.int)
    senders = np.zeros(0, dtype=np.int)
    
    for receiver, neighbor in enumerate(neighbor_indices):
      for sender in neighbor:
        # 近傍粒子から自身を除く
        if sender == receiver: continue
        receivers = np.append(receivers, receiver)
        senders = np.append(senders, sender)
    
    relative_pos = pos[receivers.astype(np.int)] - pos[senders.astype(np.int)]
    magnitude = np.linalg.norm(relative_pos, axis=1).reshape(-1, 1)
    
    e = np.hstack((relative_pos, magnitude)).astype(np.float32)
    e = torch.from_numpy(e).clone()
    e = self.epsilon_e(e)
    
    return [pos, vel, v, receivers, senders, relative_pos, magnitude, e]

  def process(self, v, receivers, senders, e, M):

    for i in range(M):
      # update edges
      de = torch.cat((e, v[receivers], v[senders]), axis=1)
      de = self.phi_e(de)
      e = e+de
  
      # update vertices
      e_idx = torch.zeros(len(v), 128)
      for idx in range(len(v)):
        e_idx[idx] = self.aggregate_e(receivers, e, idx)
      #e_idx = np.array([self.aggregate_e(receivers, e, idx).numpy() for idx in range(len(v))])
      dv = torch.cat((v, e_idx), axis=1)
      dv = self.phi_v(dv)
      v = v+dv

  def decode(self, v):
    a = self.delta_v(v)
    return a

  def aggregate_e(self, receivers, e, idx):
    value = e[receivers==idx]
    e_idx = torch.sum(value, dim=0)/len(value)
    return e_idx

# 学習

In [8]:
criterion = nn.MSELoss()
model = GraphNeuralNetwork()
optimizer = torch.optim.Adam(model.parameters())
R = 0.02

def train(model):
  data_folders = os.listdir("/content/graph_neural_network/processed")
  model.train()
  for i in range(1, 21):
    print("data: {}".format(i))
    df = pd.read_csv("/content/graph_neural_network/processed/{}/{}.csv".format(i, i))
    config = pd.read_json("/content/graph_neural_network/data/{}/{}.json".format(i, i))
    loss = 0
    flag = df[df["timestep"]==1]["label"].values == 1
    for j in range(1, config["Simulation"]["MaxTimestep"]):
      print("timestep: {}".format(j))
      df_tmp = df[df["timestep"]==j]
      a = torch.tensor(df_tmp[flag][["acc.x", "acc.y", "acc.z"]].values).float()
      pred = model(df_tmp, R, 1, config)[flag]
      loss = criterion(a, pred)
      print("loss: {}".format(loss.item()))
      loss.backward()
      optimizer.step()

In [9]:
epoch_num = 10
for epoch in range(epoch_num):
  print("-"*50)
  print("epoch: {}/{}".format(epoch+1, epoch_num))
  train(model)

ストリーミング出力は最後の 5000 行に切り捨てられました。
timestep: 73
loss: 12.434563636779785
timestep: 74
loss: 14.24359130859375
timestep: 75
loss: 14.508995056152344
timestep: 76
loss: 11.40127182006836
timestep: 77
loss: 13.104273796081543
timestep: 78
loss: 10.242894172668457
timestep: 79
loss: 9.83017349243164
timestep: 80
loss: 8.562271118164062
timestep: 81
loss: 8.290452003479004
timestep: 82
loss: 9.054256439208984
timestep: 83
loss: 7.451385974884033
timestep: 84
loss: 6.919227123260498
timestep: 85
loss: 6.03471565246582
timestep: 86
loss: 5.7066521644592285
timestep: 87
loss: 4.92777681350708
timestep: 88
loss: 5.28021240234375
timestep: 89
loss: 4.999063014984131
timestep: 90
loss: 4.9510273933410645
timestep: 91
loss: 3.911799907684326
timestep: 92
loss: 4.48929500579834
timestep: 93
loss: 4.258451461791992
timestep: 94
loss: 3.4276316165924072
timestep: 95
loss: 4.298624515533447
timestep: 96
loss: 3.3360228538513184
timestep: 97
loss: 4.165694236755371
timestep: 98
loss: 3.9192698001861572
ti

KeyboardInterrupt: ignored